In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import AzureChatOpenAI

from example.real_estate_tax_graph import small_llm

llm = AzureChatOpenAI(
    azure_deployment= 'gpt-4o-2024-11-20',
    api_version = '2024-08-01-preview',
)

small_llm = AzureChatOpenAI(
    azure_deployment= 'gpt-4o-mini-2024-07-18',
    api_version='2024-08-01-preview',
)


In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    answer: str
    context: list

graph_builder = StateGraph(AgentState)
    

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal


class Route(BaseModel):
    target: Literal['income_tax','llm','real_estate_taxh'] = Field(
        description = "The target for the query to answer"
    )

router_system_prompt = """
You are an export at routing a user's question to 'vector_store', 'llm' or 'web_search'.
'vector_store' contains information about income tax up to 2025.
'real_estate_tax' contains information about real estate tax up to 2025.
if you think the question is simple enought use 'llm'
"""

router_prompt = ChatPromptTemplate.from_messages([
    ('system', router_system_prompt),
    ('user','{query}')
])

structured_router_llm= small_llm.with_structured_output(Route)



def router(state: AgentState):
    query = state["query"]
    router_chain = router_prompt | structured_router_llm
    route = router_chain.invoke({"query": query})
    return route


In [ ]:
from langchain_core.output_parsers import StrOutputParser


def call_llm(state:AgentState):
    query = state['query']
    llm_chain = small_llm | StrOutputParser()
    llm_answer = llm_chain.invoke(query)
    return {'answer':llm_answer}

In [ ]:
from income_tax_graph import graph as income_tax_agent
from real_estate_tax_graph import graph as real_estate_tax_agent

graph_builder.add_node('income_tax', income_tax_agent)
graph_builder.add_node('real_estate_tax', real_estate_tax_agent)
graph_builder.add_node('llm', call_llm)

In [ ]:
from langgraph.graph import START, END

graph_builder.add_conditional_edges(
    START,
    router,
    {
        'income_tax': 'income_tax',
        'real_estate_tax': 'real_estate_tax',
        'llm': 'llm'
    }
)

graph_builder.add_edge('income_tax', END)
graph_builder.add_edge('real_estate_tax', END)
graph_builder.add_edge('llm', END)


In [ ]:
graph = graph_builder.compile()

In [ ]:
initial_state = {'query': '소득세란 무엇인가?'}

graph.invoke(initial_state)


In [ ]:
initial_state = {'query': '5억짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 때 세금을 얼마나 내나요?'}

graph.invoke(initial_state)


In [ ]:
initial_state = {'query': '떡볶이는 어디가 맛있나요??'}

graph.invoke(initial_state)